# **March Madness Machine Learning 2025** #
##### By: CoNiya Butler & Daniel Davis #####

### **Introduction** ###

This notebook explores historical NCAA Division I men's and women's basketball data to predict the outcomes of March Madness tournament games. We'll leverage provided datasets containing team information, game results, and tournament seeds to build a predictive model. Data files are prefixed with 'M' for men's, 'W' for women's,and some span both.

##### Goal: #####


Minimize the Brier score, the evaluation metric for this competition.


##### Approach: #####

1. Start with a model with the features on the list provided below.
    - Seeding differences
    - Average points Per Game
    - Win percentage regular season
    - Win percentage tournament
    - Location
    - Average scoring difference per game
2. Apply feature engineering and model tuning techniques

### **Import Libraries** ###

In [2]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import log_loss, brier_score_loss, mean_squared_error, roc_curve, auc
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, accuracy_score
import warnings
warnings.filterwarnings("ignore")

### **Load Data** ###

Load the datasets need for the model M is for men's basketball and W is for women's basketball

In [ ]:
M_seed_df = pd.read_csv("/app/March-Madness-Prediction/Data/march-machine-learning-mania-2025/MNCAATourneySeeds.csv")
W_seed_df = pd.read_csv("/app/March-Madness-Prediction/Data/march-machine-learning-mania-2025/WNCAATourneySeeds.csv")
M_regular_results = pd.read_csv("/app/March-Madness-Prediction/Data/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv")
M_tourney_results = pd.read_csv("/app/March-Madness-Prediction/Data/march-machine-learning-mania-2025/MNCAATourneyDetailedResults.csv")
W_regular_results = pd.read_csv("/app/March-Madness-Prediction/Data/march-machine-learning-mania-2025/WRegularSeasonDetailedResults.csv")
W_tourney_results = pd.read_csv("/app/March-Madness-Prediction/Data/march-machine-learning-mania-2025/WNCAATourneyDetailedResults.csv")

### **Data Preprocessing** ###

In [9]:
# Join the men's and women's dataset together
regular_results = pd.concat([M_regular_results, W_regular_results])
tourney_results = pd.concat([M_tourney_results, W_tourney_results])
seed_df = pd.concat([M_seed_df, W_seed_df])

### **Exploratory Data Analysis (EDA)** ###

In [11]:
seed_df['Seed'].unique()

array(['W01', 'W02', 'W03', 'W04', 'W05', 'W06', 'W07', 'W08', 'W09',
       'W10', 'W11', 'W12', 'W13', 'W14', 'W15', 'W16', 'X01', 'X02',
       'X03', 'X04', 'X05', 'X06', 'X07', 'X08', 'X09', 'X10', 'X11',
       'X12', 'X13', 'X14', 'X15', 'X16', 'Y01', 'Y02', 'Y03', 'Y04',
       'Y05', 'Y06', 'Y07', 'Y08', 'Y09', 'Y10', 'Y11', 'Y12', 'Y13',
       'Y14', 'Y15', 'Y16', 'Z01', 'Z02', 'Z03', 'Z04', 'Z05', 'Z06',
       'Z07', 'Z08', 'Z09', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15',
       'Z16', 'Y16a', 'Y16b', 'W16a', 'W16b', 'X16a', 'X16b', 'Z16a',
       'Z16b', 'W12a', 'W12b', 'Z11a', 'Z11b', 'X12a', 'X12b', 'Z14a',
       'Z14b', 'Y11a', 'Y11b', 'Z13a', 'Z13b', 'Y12a', 'Y12b', 'W11a',
       'W11b', 'X11a', 'X11b', 'Y10a', 'Y10b', 'Z10a', 'Z10b', 'Z12a',
       'Z12b'], dtype=object)

### **Model Training** ###

### **Prediction Submission** ###